In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL = not IN_COLAB
USE_GITHUB = True
USE_DRIVE = False
device = 'cuda'

assert not (USE_GITHUB and USE_DRIVE)

In [3]:
if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  if USE_GITHUB:
    !git clone https://github.com/kejeon/in_dev_RN20Q.git
    %cd '/content/in_dev_RN20Q'
  elif USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

fatal: destination path 'in_dev_RN20Q' already exists and is not an empty directory.
/content/in_dev_RN20Q


wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')
state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device(device))

teacher_model = resnet20()

try:
  teacher_model.load_state_dict(state_dict['net'])
except:
  # if the model is wrapped in a module, update all keys in state_dict to remove module.
  state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}
  teacher_model.load_state_dict(state_dict['net'])

wandb:   1 of 1 files downloaded.  


In [ ]:
lr = 0.02
arch_tag = "ResNet20_Q"
batch_size = 128
dataset = "CIFAR10"
T=20
alpha=0.7

student_model = ResNet20_Q(a_bit=4, w_bit=1)

run = wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet20_Q_1bit",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": arch_tag,
    "batch_size": batch_size,
    "dataset": dataset,
    "T": T,
    "alpha": alpha,
    "scheduler": 'MultiStepLR',
    "style": 'with_KD'
    }
)

my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           T=T,
                           alpha=alpha,
                           device=device,
                           batch_size=batch_size,
                           lr=lr)

my_trainer.train_script(200)

wandb.finish()

runtime.unassign()

Files already downloaded and verified
Files already downloaded and verified

Epoch: 0


Epoch 0:   0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
Epoch 0: 100%|██████████| 391/391 [00:24<00:00, 16.08it/s, train_acc=0.187, xentropy=1.489]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:437: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


test_acc: 0.244
Saving..

Epoch: 1


Epoch 1: 100%|██████████| 391/391 [00:20<00:00, 18.72it/s, train_acc=0.275, xentropy=1.352]


test_acc: 0.229

Epoch: 2


...

Epoch 84: 100%|██████████| 391/391 [00:20<00:00, 18.67it/s, train_acc=0.748, xentropy=0.580]


test_acc: 0.693

Epoch: 85


Epoch 85: 100%|██████████| 391/391 [00:20<00:00, 18.77it/s, train_acc=0.748, xentropy=0.579]


test_acc: 0.714

Epoch: 86


Epoch 86:  60%|█████▉    | 233/391 [00:12<00:07, 20.45it/s, train_acc=0.748, xentropy=0.581]